**This program will parse your data of midi files into data readable by the ai**

Import

In [5]:
!pip install py_midicsv
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.6 MB/s eta 0:00:00


In [10]:
!rm -rf /content/music/

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [11]:
!unzip /content/music.zip -d /content/music/

Archive:  /content/music.zip
  inflating: /content/music1k/(FF1Eriestg2).mid  
  inflating: /content/music1k/(FFT2)GM(R2).mid  
  inflating: /content/music1k/(FFT2)R2).mid  
  inflating: /content/music1k/1516.mid  
  inflating: /content/music1k/1812_overture_gr.mid  
  inflating: /content/music1k/2_01_FFVII_Main_Theme.mid  
  inflating: /content/music1k/2-01 FFVII Main Theme Awe.mid  
  inflating: /content/music1k/2-01 FFVII Main Theme Ygm.mid  
  inflating: /content/music1k/72-06-supper1.mid  
  inflating: /content/music1k/72-06-supper2xg.mid  
  inflating: /content/music1k/9beethoven4mvto.mid  
  inflating: /content/music1k/adore3.mid  
  inflating: /content/music1k/ak_ff5final.mid  
  inflating: /content/music1k/ak_ff5new.mid  
  inflating: /content/music1k/all time 7.mid  
  inflating: /content/music1k/B5-4gus.mid  
  inflating: /content/music1k/B5th_5g_1.mid  
  inflating: /content/music1k/balamb-ff8.mid  
  inflating: /content/music1k/beatmaster_backonceagain.mid  
  inflating: /

**Convert the midis into all one track (You need to manually make the directory)**

In [1]:
import os
import mido

def merge_midi_tracks(input_file):
    try:
      mid = mido.MidiFile("/content/music/" + input_file)

      merged = mido.MidiFile(ticks_per_beat=mid.ticks_per_beat)

      # Create a new track to merge the MIDI events
      merged_track = mido.MidiTrack()
      merged.tracks.append(mido.merge_tracks(mid.tracks))

      # Save the merged MIDI file

      merged.save("/content/musicFixed/" + input_file)
      print(input_file)
    except:
      pass

for filename in os.listdir("/content/music/"):
    if filename.endswith('.mid'):
      merge_midi_tracks(filename)

ff8dollt.mid
ff7-Hurry.mid
ff3ending.mid
ffxii_entrance.mid
ff7-Crazy_Motorcycle.mid
fftworry.mid
ff11battle1-1.mid
send weebs to war 0.9.mid
ff8heres.mid
ff2 - TheLandOfDwarves.mid
ff7_2-19.mid
(FFT2)GM(R2).mid
ff8-Slide_Show_Part2.mid
ff3btlre.mid
ff311.mid
ff5dung.mid
ffx_isnt_it_beautiful_guitar.mid
ff5_1_11.mid
ff5home.mid
FF7_FinalDungeon.mid
FF5_Unknown_Lands-KM-GM.mid
Ffname.mid
(FFT2)R2).mid
ff8-maybe_i'm_a_lion-2.mid
ff9zidan.mid
ff5Intro.mid
ff8rw.mid
ff7mid102.mid
ff9-Opening-long_remix.mid
ff10-2_YuRiPa_Battle_1.mid
xartlife4.mid
jhh38-ff-faf.mid
ff9chosensummoner.mid
ff7boss (2).mid
ff7jenova_4.mid
ff7mid109.mid
ff9-memories.mid
FFVI_-_The_Unforgiven.mid
ff356.mid
ff9amara.mid
fftattack.mid
FFVIII_Ending_Part_II_S-YXG50.mid
FF7_Fanfare-KM-GS.mid
ff321.mid
ff4-Golbeza_Clad_in_the_Dark.mid
ff8-Force_Your_Way.mid
TerraV2_by_Mister_FF.mid
ff3strgo.mid
ff9pand_V1_0_1.mid
ff8_battle_techno.mid
FFX-2-Real_Emotion.mid
ff8-A_Sacrifice.mid
ff7-Wutai.mid
ff2.mid
ff2prologue.mid
ff8u

In [2]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/musicFixed/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  outValue = str(note)

  if(instrument<=127):
    outValue += "%"

  outValue += str(start)

  if(velocity > 108):
    outValue += "$"
  elif velocity > 74:
    outValue += "#"
  elif velocity > 40:
    outValue += "@"
  else:
    outValue += "!"

  outValue += str(length)

  return outValue

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
for midi_file in midi_files:
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    csv_parsed = []
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

for i in range(0, len(csv_string), 1800):
  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

Event length mismatch for PortEvent at position 188
Event length mismatch for PortEvent at position 27
Event length mismatch for PortEvent at position 443
Event length mismatch for PortEvent at position 677
Event length mismatch for PortEvent at position 48
Event length mismatch for PortEvent at position 476
Event length mismatch for PortEvent at position 55
Event length mismatch for PortEvent at position 55
Event length mismatch for PortEvent at position 676
Event length mismatch for PortEvent at position 70
Event length mismatch for PortEvent at position 144
Event length mismatch for PortEvent at position 58
Event length mismatch for PortEvent at position 56
Event length mismatch for PortEvent at position 216
Event length mismatch for PortEvent at position 263
Event length mismatch for PortEvent at position 199
Event length mismatch for PortEvent at position 56
Event length mismatch for PortEvent at position 3944
Event length mismatch for PortEvent at position 70
Event length mismatc

#Ignore this

In [41]:
midi_header = '''0, 0, Header, 1, 11, 480
1, 0, Start_track
1, 0, Title_t, ""
1, 0, Time_signature, 4, 2, 24, 8
1, 0, Tempo, 324324
1, 102960, End_track
2, 0, Start_track
2, 0, Control_c, 0, 121, 0
2, 0, Title_t, ""
2, 0, Control_c, 0, 10, 64
2, 0, Control_c, 0, 7, 100
2, 0, Control_c, 0, 11, 127
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 2
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 1
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 38, 0
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 0
2, 0, Control_c, 0, 6, 12
2, 0, Pitch_bend_c, 0, 8192
2, 0, Control_c, 0, 1, 0
2, 0, Program_c, 0, 57
'''

In [49]:
def generate_on_off_events(noteData):
  track_num = "2"
  #TODO:
  dynamic = 127
  note_on = track_num + ", " + str(noteData[2]) + ", Note_on_c, 0, " + str(noteData[0]) + ", " + str(dynamic)
  note_off = track_num + ", " + str(noteData[2] + noteData[4]) + ", Note_off_c, 0, " + str(noteData[0]) + ", " + "0"
  return [[noteData[2], note_on], [noteData[2] + noteData[4], note_off]]


def test_if_num(num):
  try:
    tmp = int(num)
  except:
    return False

  return True

newCsv = midi_header

file = open("test.txt", "r").read()
fileTokens = file.split("|")
allParsedVals = []

timeMarker = 0
for i in range(len(fileTokens)):
  valuesArr = []
  valueBuild = ""
  for j in range(len(fileTokens[i])):
    if(test_if_num(fileTokens[i][j]) or fileTokens[i][j] == "-"):
      valueBuild += fileTokens[i][j]
    else:
      print(valueBuild)
      valuesArr.append(int(valueBuild))
      valuesArr.append(fileTokens[i][j])
      valueBuild = ""
  if(valueBuild != ""):
    valuesArr.append(int(valueBuild))
    valueBuild = ""

  oldTime = valuesArr[2]
  valuesArr[2] += timeMarker
  allParsedVals.append(valuesArr)
  timeMarker += oldTime

noteEventsArray = []
for val in allParsedVals:
  events = generate_on_off_events(val)
  noteEventsArray.append(events[0])
  noteEventsArray.append(events[1])
noteEventsArray = sorted(noteEventsArray, key=lambda x: x[0])

outString = midi_header
for val in noteEventsArray:
  outString += val[1] + "\n"
outString += '''2, ''' + str(noteEventsArray[len(noteEventsArray) - 1][0] + 1000) + ''', End_track
0, 0, End_of_file'''

with open('mididata.csv', 'w') as file:
    file.write(outString)

42
12
51
0
43
15
52
0
43
15
53
0
44
15
54
0
59
15
58
0
53
15
59
0
59
15
58
0
53
15
59
0
59
15
58
0
53
15
59
0
57
15
53
0
57
15
59
0
59
15
58
0
53
15
59
0
57
15
54
0
59
15
59
0
58
15
53
0
60
-8
15
51
15
57
0
59
15
58
0
53
15
59
0
57
15
53
0
56
-8
15
51
15
56
0
59
15
57
0
59
15
30


In [50]:
import py_midicsv as pm

csv_string = "/content/mididata.csv"

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("aiOut.mid", "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)